# Introducción a Julia 2

## Estructuras

Hemos visto que **todo**, realmente **TODO**, en Julia tiene asociado un tipo o estructura. Aquí veremos varias formas de crear nuevos tipos que se acomoden a lo que necesitamos, y algunos trucos para que la ejecución sea rápida. La importancia de los tipos radica, como vimos, en el hecho que la elección de qué método de una función se usa depende del tipo de sus argumentos.

La convención a la hora de definir tipos es que estén escritos en estilo "camello", es decir, en que la primer letra de cada palabra empieza en mayúscula. Por ejemplo, tenemos `Float64`, `AbstractFloat`.

Es importante decir que los tipos **no** pueden ser redefinidos o sobreescritos en una sesión de Julia "normal"; para hacerlo, hay que iniciar una nueva sesión o reiniciar el kernel del notebook (para el Jupyter notebook).

### Tipos inmutables y constructores internos

Crearemos por ahora una estructura "vacía", simplemente para ilustrar cómo se definen las estructuras y cómo se definen los constructores.


In [ ]:
# Definimos la estructura vacía "MiTipo"
struct MiTipo
end

Para crear un objeto del tipo `MiTipo` se requiere un *constructor*, que sencillamente es una función que devuelve un objeto del tipo especificado. Julia, por default, se encarga de tener dicha función una vez que la estructura ha sido definida.

In [ ]:
methods(MiTipo)

La estructura `MiTipo` que acabamos de definir **no** contiene ningún tipo de datos, por lo que se llama "singleton". Este tipo de estructuras pueden ser útiles para cuestiones de *dispatch*, esto es, de distinguir el método que se usa. (Esto se aclarará a su debido tiempo.)

In [ ]:
mt = MiTipo()

In [ ]:
typeof(mt)

In [ ]:
mt isa MiTipo # equivale a `isa(mt, MiTipo)`

En general, cuando definimos un tipo nuevo es para que contenga cierto tipo de datos, que por una u otra razón tienen un significado importante para el problema que lo motiva.

La siguiente estructura define a `Partic1d`, que podría representar la posición y velocidad 
de una partrícula en 1 dimensión.


In [ ]:
struct Partic1d
    x :: Float64
    v :: Float64
end

Por cuestiones de eficiencia es conveniente que los tipos de las variables *internas* sean concretos; en este caso hablamos de `x` y `v`. Si se requiere aún más versatilidad respecto a los tipos internos, se puede definir *tipos parámetricos*, como veremos más adelante.

Es importante enfatizar que las distintas componentes internas de un tipo pueden tener distintos tipos asociados, por ejemplo, `Float64` y `String`.

Para acceder a la información de los campos internos de un tipo, usamos la función `fieldnames`:


In [ ]:
fieldnames(Partic1d)

El método que por default crea a un objeto tipo `Partic1d` requiere que especifiquemos *en el mismo orden* todos los *campos* que lo componen.

In [ ]:
methods(Partic1d)

In [ ]:
p1 = Partic1d(1.0, -2.4)

In [ ]:
p1.x # accede al *valor* del campo `x`

In [ ]:
getfield(p1, :v) # Otra manera de obtener el campo `:v` de p1

In [ ]:
Partic1d(1, pi)

El tipo de estructura que acabamos de crear es *inmutable*, lo que significa que los campos individuales (cuando son *concretos*), no se pueden cambiar. Esto lo que significa es que si tratamos de cambiar el campo interno de un tipo inmutable, Julia arrojará un error.


In [ ]:
isimmutable(p1)

In [ ]:
p1.x = 2.0

La propiedad de inmutabilidad no es recursiva; esto es, si una componente de un tipo consiste de algún campo que es mutable (por ejemplo, `Array{T,N}`), entonces las componentes individuales de ese campo pueden cambiar.

In [ ]:
struct Partic2d
    pos :: Array{Float64,1}
    vel :: Vector{Float64}
    #La siguiente función se llama constructor interno
    function Partic2d(x::Array{Float64,1}, v::Array{Float64,1})
        @assert length(x) == length(v) == 2
        return new(x, v)
    end
end

La función que aparece en el interior redefine el constructor de default, y se llama *constructor interno*. Hay que enfatizar que el comando `new` *sólo* se utiliza en el caso de constructores internos; en algún sentido estamos devolviendo un objeto tipo `Partric2d`, que aún no está definido.

In [ ]:
p2 = Partic2d([1.0, 2.5], [1.0, 3.0])

In [ ]:
fieldnames(Partic2d)

Como dijimos antes, no se puede cambiar *el objeto en si* (por ser inmutable), pero sí sus componentes.

In [ ]:
p2.pos = [2, 1]  # Arroja un error

In [ ]:
p2.pos[1] = 6.0  # cambia la primer componente del campo `:x`

In [ ]:
p2.pos .= [2, 1] # Cambiamos componente a componente (con broadcasting) todo el campo `:x`

In [ ]:
p2

### Tipos mutables

Todo lo dicho anteriormente se puede extender para definir tipos mutables. La única diferencia es a la hora de definirlos: debemos usar `mutable struct`.

In [ ]:
mutable struct MPartic2d
    x :: Array{Float64,1}
    v :: Array{Float64,1}
    function MPartic2d(x :: Array{Float64,1}, v :: Array{Float64,1})
        @assert length(x) == length(v) == 2
        return new(x, v)
    end
end

In [ ]:
mp2 = MPartic2d([1.0, 2.5], [1.0, 3.0])

In [ ]:
mp2.x = [2, 1]  # Funciona, ya que el tipo es mutable!

In [ ]:
mp2

### Estructuras paramétricas

En ocasiones uno quiere definir estructuras que operen con distinto tipo de entradas. Un ejemplo son los racionales: tenemos `Rational{Int64}` y *también* `Rational{BigInt}`; otro ejemplo son los complejos: `Complex{Int64}` o `ComplexF64`, que es un alias de `Complex{Float64}`.

Anteriormente, definimos `Partic2d` con campos que son vectores `Array{Float64,1}`, por lo que usar otro tipo de vectores arroja un error.

In [ ]:
Partic2d([1, 2], [1, 3])

En principio uno *podría* usar en la definición de los campos que componen al tipo, tipos abstractos, como `Real`. Sin embargo, dado que el compilador *no* conoce la estructura en memoria de tipos abstractos de antemano, el código que se ejecutará eventualmente será ineficiente. Un ejemplo de código ineficiente, entonces, sería:
```julia
#Estructura MUY ineficiente
struct Partic3dIneficiente
    x :: Array{Real,1}
    v :: Array{Real,1}
end
```
dado que `Real` es un tipo abstracto.


La alternativa es definir estructuras *paramétricas*, donde precisamente el parámetro es un tipo concreto (sin especificar) que es subtipo de algún tipo abstracto.

In [ ]:
struct Partic3d{T<:Real}
    x :: Array{T,1}
    v :: Vector{T}
    function Partic3d(x :: Array{T,1}, v :: Array{T,1}) where {T<:Real}
        @assert length(x) == length(v) == 3
        return new{T}(x, v)
    end
end

En cierto sentido, en la definición anterior de `Partic3d{T}` la `T` adquiere un tipo concreto, que es subtipo de `Real`, y que es el que se utiliza en los campos donde se requiere especificar dentro del constructor.

In [ ]:
Partic3d([1//1,2,3], [2,3,4//2])  # regresa un Partic3d{Int}

In [ ]:
Partic3d([1.5,2,3], [2.5,3,4]) # regresa un Partic3d{Float64}

Los tipos están organizados en una estructura de árbol; en todos los casos anteriores, la definición los ha puesto directamente abajo de `Any`.

In [ ]:
supertype(Partic3d)

In [ ]:
supertype(Float64)

In [ ]:
supertype(AbstractFloat)

In [ ]:
subtypes(Real)

Uno puede de hecho insertar en cualquier punto del árbol de tipos los tipos definidos. Esto es útil porque permite obtener cierta clase de sobrecarga de operadores, y por lo mismo, la posibilidad de aplicar ciertas funciones a la estructura que hemos creado.

El siguiente ejemplo define la estructura paramétrica `MiVector2d`, y la pone como subtipo de `AbstractArray`; noten que `AbstractArray` *también* es una estructura paramétrica.

In [ ]:
struct MiVector2d{T<:Real} <: AbstractArray{T,1}
    x :: T
    y :: T
end

In [ ]:
x = MiVector2d(1, 2) # da un error !?

El error indica *algo* no relacionado con lo que hemos hecho, sino que tiene que ver con la visualización de `x`. (El mensaje dice que el problema está con `size`.) Uno puede notar que `x.x` y `x.y` dan los resultado esperados; de hecho, `x` ha sido *definido*, simplemente, no lo podemos visualizar.

In [ ]:
isdefined(Main, :x)

In [ ]:
x.x, x.y

Para hacernos la vida más sencilla a la hora de visualizar `MiVector2d`, sobrecargaremos `size` y `getindex`.

In [ ]:
size([1,2,3,5])

In [ ]:
@which size([1,2])

In [ ]:
import Base: size
size(::MiVector2d{T}) where {T} = (2,) # dos componentes en la primer dimensión
# size(::MiVector2d) = (2,) # dos componentes en la primer dimensión

In [ ]:
x

In [ ]:
function Base.getindex(v::MiVector2d, i::Int)
    if i == 1
        return v.x
    elseif i == 2
        return v.y
    else
        throw(AssertError)  # Esto "dispara" un error tipo `AssertError`
    end
end

In [ ]:
x

In [ ]:
y = MiVector2d(1.2, 2.1)

A pesar de que **no** hemos sobrecargado la suma (`:+`), ésta funciona gracias a la estructura de tipo que hemos impuesto a `MiVector2d{T} <: AbstractArray{T,1}`.

In [ ]:
x + y

Sin embargo, hay que notar que el resultado es un `Array{Float64,1}` y no un `MiVector2d{Float64}`. Para logra que el resultado sea del tipo que queremos, debemos sobrecargar la función `:+`.

In [ ]:
((x .+ y)...,)

In [ ]:
Base.:+(x::MiVector2d, y::MiVector2d) = MiVector2d((x .+ y)...)

In [ ]:
x + y

Este ejemplo *no* es uno particularmente interesante, pero muestra que Julia permite adecuar las cosas a lo que requerimos, y *extender* las funciones de Julia para que la interacción sea sencilla y cómoda.

Para que `MiVector2d` funcione con entradas de vector, debo definir un constructor apropiado:

In [ ]:
MiVector2d(v::Vector{<:Real}) = MiVector2d(v...,)

In [ ]:
MiVector2d([1,2])

## Metaprogramming

### Expresiones

Julia, igual que Lisp, representa al código (por ejemplo, en el REPL) como una estructura de datos en el *propio lenguaje*. Entonces, es posible escribir y modificar código de manera programática. La posibilidad de escribir código que genere y modifique código es lo que se entiende por "Metaprogramming".

Aquí ilustraremos algunos conceptos, siguiendo el [manual](https://docs.julialang.org/en/v1/manual/metaprogramming), dejando varios temas sin cubrir.

In [ ]:
1+1

In [ ]:
# Cualquier línea de código inicialmente es una cadena:
prog = "1 + 1"

In [ ]:
# El siguiente paso es convertir la cadena en una expresión:
ex1 = Meta.parse(prog)

In [ ]:
typeof(ex1)

In [ ]:
propertynames(ex1)

In [ ]:
fieldnames(Expr)

Claramente, un objeto tipo Expr tiene dos campos. Primero, tenemos `head`, que es un `Symbol` que define el tipo de expresión. En este caso se trata de un `:call`.

In [ ]:
ex1.head

Por otro lado tenemos `args`, que es un `Vector{Any}` y contiene los argumentos de la expresión

In [ ]:
typeof(ex1.args)

In [ ]:
ex1.args

Las expresiones también pueden ser escritas directamente a partir del constructor de un objeto tipo `Expr`:

In [ ]:
ex2 = Expr(:call, :+, 1, 1) # equivalente a Expr(:call, [:+, 1, 1])

In [ ]:
ex1 == ex2

El punto importante es que el código en Julia está representado internamente por expresiones escritas en Julia y que son accesibles desde Julia.

La función `dump()`` da información anotada de la expresión:

In [ ]:
dump(ex1)

In [ ]:
+(1,1)

Expresiones más complejas se construyen de forma similar:

In [ ]:
ex3 = Meta.parse("(4 + sin(1.0)) / 2")

Otra manera de visualizar a la expresión es con `Meta.show_sexpr`

In [ ]:
Meta.show_sexpr(ex3)

Uno de los usos de `:` es crear símbolos, o también se puede usar `Symbol()`

In [ ]:
:foo == Symbol("foo")

`Symbol` permite concatenar distintas partes, que esencialmente se toman como
cadenas

In [ ]:
Symbol(:var,'_',"sym",3)

Otro uso de `:` es crear expresiones sin usar el constructor `Expr`, en lo que
se llama *citar* (en inglés, *quoting*)

In [ ]:
ex4 = :(a + b*c + 1)

In [ ]:
dump(ex4)

No sólo es el hecho de que podamos escribir programáticamente las expresiones, sino que
también podemos modificarlas. Como ejemplo, tomaremos `ex4`, y la transformaremos
de ser `:(a + b * c + 1)` a ser `:(a + b * c + 2.1)`. Esto, simplemente lo
conseguimos cambiando el cuarto elemento del vector `ex4.args`:

In [ ]:
ex4.args[4] = 2.1

In [ ]:
ex4

Otra manera de construir expresiones más complejas es usando el bloque `quote ... end`

In [ ]:
ex = quote
    xx = 1
    yy = 2
    xx + yy
end

Para evaluar una expresión, es decir, considerar a la cadena de texto y *correrla*, se utiliza la función `eval`. En la expresión anterior, las variables `xx` y `yy` no han sido evaluadas, y por eso se obtienen `UndefVarError`.

In [ ]:
xx

In [ ]:
eval(ex)

In [ ]:
xx, yy

Las expresiones pueden involucrar variables cuyo valor ha sido asignado; evaluar dichas
expresiones utiliza el valor de estas variables:

In [ ]:
z = 4
eval( :(2*xx + z) )

Incluso, uno puede *sustituir* el valor de esas variables, usando `$`, de la misma
manera que uno *interpola* valores en cadenas

In [ ]:
dump( :(2*xx + $z) )

### Generación de código

Un ejemplo un poco más interesante, es el implementar la evaluación de los polinomios
de Wilkinson:
$$
W_n(x) = \prod_{i=1}^{n} (x-i) = (x-1)(x-2)\dots(x-n).
$$

In [ ]:
nombre(n::Int) = Symbol( string("W_", n) )

In [ ]:
nombre(3)

La siguiente función regresa la expresión que corresponde al polinomio de Wilkinson
$W_n(x)$.

In [ ]:
function wilkinson(n::Int)
    # Imponemos que que `n` sea ≥ 1
    @assert n ≥ 1 "`n` tiene que ser mayor o igual a 1"

    ex = :(x-1)
    for i = 2:n
        ex = :( ($ex) * ( x-$i) )
    end
    ex_ret = :( $(nombre(n))(x) = $ex )
    return ex_ret
end

In [ ]:
wilkinson(0) # Da un AssertionError !

In [ ]:
w3 = wilkinson(3)
eval(w3)

In [ ]:
W_3(2.1)

Uno puede *automatizar* la generación de código. Tomando el ejemplo de los
polinomios de Wilkinson podemos, dentro de un ciclo `for`, generar varios
de éstos.

In [ ]:
for i = 1:10
    ex = wilkinson(i) # genera el polinomio de orden `i`
    println(ex)
    @eval $ex
end

In [ ]:
W_8(1.0)

Esta forma de generar código permite tener código más conciso y sencillo de
mantener, aunque debe ser utilizado con cuidado.

### Macros

En ocasiones hemos usado instrucciones que incluyen `@` antes de la *expresión*,
un ejemplo es `@assert`. Éstos son macros: Los macros son funciones cuyas
entradas son expresiones, que son manipuladas y al final se evalúan.

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    return 0     # for simplicity
end

In [ ]:
@simple_example(x+y)

In [ ]:
# Cambiemos un poco el macro `@simple_example`
macro simple_example(expr)
    @show expr   # this is another macro !
    return expr     # for simplicity
end

In [ ]:
x, y

In [ ]:
@simple_example x + y

In [ ]:
@simple_example x1 + y1

El macro `@macroexpand` permite ver lo que hace el macro:

In [ ]:
@macroexpand @simple_example x1 + y1

Una sutileza importante de los macros es que, a diferencia de las funciones,
los macros permiten introducir y modificar código *antes* de que sea
ejecutado, dado que los macros son ejecutados cuando el código se traduce
en expresiones (*parse time*).

El siguiente ejemplo, tomado del manual, ilustra esto:

In [ ]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)

    return :(println("I execute at runtime. The argument is: ", $arg))
end

In [ ]:
exx = macroexpand(Main, :(@twostep :(1, 2, 3)) );

El primer uso de `println` ocurre cuando `macroexpand` es utilizado; la expresión resultante incluye el segundo `println` únicamente.

In [ ]:
exx

In [ ]:
eval(exx)

Más información sobre los macros puede ser encontrada
[aquí](https://docs.julialang.org/en/v1/manual/metaprogramming), e incluye ejemplos
de [generación de código](https://docs.julialang.org/en/v1/manual/metaprogramming/#Code-Generation) que
son útiles, [cadenas literales no estándar](https://docs.julialang.org/en/v1/manual/metaprogramming/#meta-non-standard-string-literals)
o [funciones generadoras](https://docs.julialang.org/en/v1/manual/metaprogramming/#Generated-functions).
La lectura de este capítulo del manual es altamente recomendada.